In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

# import sys; sys.path.insert(0, r'C:\Users\Lukas\Documents\projects\invert')
import sys; sys.path.insert(0, r'C:\Users\Lukas\OneDrive\Dokumente\projects\invert')
import os
import mne
import pickle as pkl
from time import time
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from invert.forward import get_info, create_forward_model
from invert.util import pos_from_forward

os.makedirs("figures", exist_ok=True)
# with low contrast
pp = dict(surface='inflated', hemi='both', verbose=0, cortex='low_contrast')
order = ["AP", "RAP-MUSIC", "FLEX-AP", "FLEX-RAP-MUSIC", "Convexity Champagne", "eLORETA", "LCMV"]
# estimator = "median"
estimator = "mean"
errorbar = ("ci", 95)
# order = ["AP", "RAP-MUSIC", "FLEX-AP", "FLEX-RAP-MUSIC"]

In [2]:
def get_pos(fwd):
    pos_left = mne.vertex_to_mni(fwd["src"][0]["vertno"], 0, subject=subject, subjects_dir=subjects_dir, verbose=0)
    pos_right = mne.vertex_to_mni(fwd["src"][1]["vertno"], 1, subject=subject, subjects_dir=subjects_dir, verbose=0)
    pos = np.concatenate([pos_left, pos_right], axis=0)
    return pos

path = r"D:\data\flex_ssm\evaluation\sim_and_preds_figure-42_1-assumed_Rotation-Right-2_greedy.pkl"
with open(path, "rb") as file:
    stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply = pkl.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\data\\flex_ssm\\evaluation\\sim_and_preds_figure-42_1-assumed_Rotation-Right-2_greedy.pkl'

In [9]:
import os
from invert.evaluate import eval_mean_localization_error

idx = -1

y_true = y_test[idx].toarray().T


fullpath = os.path.join("D:/data/flex_ssm", forward_models[0]["path_fwd"])
fwd_orig = mne.read_forward_solution(fullpath, verbose=0)
fwd_orig = mne.convert_forward_solution(fwd_orig, surf_ori=True, force_fixed=True, use_cps=True, verbose=0)
adjacency_orig = mne.spatial_src_adjacency(fwd_orig["src"], verbose=0).toarray()
pos_orig = get_pos(fwd_orig)

fullpath = os.path.join("D:/data/flex_ssm", forward_models[3]["path_fwd"])
fwd = mne.read_forward_solution(fullpath, verbose=0)
fwd = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True, use_cps=True, verbose=0)
adjacency = mne.spatial_src_adjacency(fwd["src"], verbose=0).toarray()
pos = get_pos(fwd)

distance_matrix = cdist(pos_orig, pos)

# True
stc = mne.SourceEstimate(data=np.zeros((fwd_orig['nsource'], 1)), vertices=[fwd_orig['src'][0]['vertno'], fwd_orig['src'][1]['vertno']], tmin=0, tstep=1, subject="fsaverage")
stc.data = y_true.T
stc.plot(**pp)

for solver_name in stc_dict.keys():
    y_est = stc_dict[solver_name][idx].toarray()
    stc = mne.SourceEstimate(data=np.zeros((fwd['nsource'], 1)), vertices=[fwd['src'][0]['vertno'], fwd['src'][1]['vertno']], tmin=0, tstep=1, subject="fsaverage")
    stc.data = y_est
    stc.plot(**pp, brain_kwargs=dict(title=solver_name))

    mle = eval_mean_localization_error(y_true.T, y_est, adjacency_orig, adjacency, distance_matrix, mode="match")
    mle_amir = eval_mean_localization_error(y_true.T, y_est, adjacency_orig, adjacency, distance_matrix, mode="amir")
    print(f"{solver_name}: {mle:.2f} mm")
    if mle / mle_amir > 1.01 or mle / mle_amir < 0.99:
        print(f"\t{solver_name} (amir): {mle_amir:.2f} mm (off by {mle - mle_amir:.5f} mm)")


C:\Users\lukas\AppData\Local\Temp\ipykernel_2680\2028470544.py:33: RuntimeWarning: All data were zero
  stc.plot(**pp, brain_kwargs=dict(title=solver_name))


SSM: 3.45 mm


C:\Users\lukas\AppData\Local\Temp\ipykernel_2680\2028470544.py:33: RuntimeWarning: All data were zero
  stc.plot(**pp, brain_kwargs=dict(title=solver_name))


AP: 3.53 mm


C:\Users\lukas\AppData\Local\Temp\ipykernel_2680\2028470544.py:33: RuntimeWarning: All data were zero
  stc.plot(**pp, brain_kwargs=dict(title=solver_name))


RAP-MUSIC: 7.92 mm


In [35]:
from invert.simulate import generator_single_simple, generator_simple


leadfield = fwd["sol"]["data"]
corr = 0.5
T = 50
nsources = 2
indices = np.array([0,1])
SNR = 0

generator_single_simple(leadfield, corr, T, nsources, indices, SNR)

batch_size = 32
corrs = (0.49, 0.50)
nsources = 2
SNR_range = (0, 10)
generator = generator_simple(fwd, batch_size, corrs, T, nsources, SNR_range, random_seed=42)
x, y = generator.__next__()
print(x.shape, y.shape)

stc = mne.SourceEstimate(data=y[0], vertices=[fwd['src'][0]['vertno'], fwd['src'][1]['vertno']], tmin=0, tstep=1, subject="fsaverage")
stc.plot(**pp)

(32, 305, 50) (32, 8196, 50)


C:\Users\lukas\AppData\Local\Temp\ipykernel_7884\4274535830.py:22: RuntimeWarning: All data were zero
  stc.plot(**pp)


In [36]:
# np.where(y[0, :, 0]!=0)
from scipy.stats import pearsonr
pearsonr(y[0, 4709, :], y[0, 5202, :])

PearsonRResult(statistic=0.9999999999999999, pvalue=0.0)

In [45]:
from scipy.stats import pearsonr
corrs = []
corrs_random = []
n = 1000
Q = 2
T = 1000
for i in range(n):
    freq = np.random.randint(10, 31, Q)  # random frequencies between 10Hz to 30Hz
    
    phases = 2 * np.pi * np.random.rand(Q)  # random phases
    t = np.linspace(10 * np.pi / T, 10 * np.pi, T)
    Signals = np.sqrt(2) * np.cos(2 * np.pi * freq[:, None] * t + phases[:, None])  # the basic signals
    corr = pearsonr(Signals[0, :], Signals[1, :])[0]
    corrs.append(corr)

    corrs_random.append(pearsonr(np.random.randn(T), np.random.randn(T))[0])

import seaborn as sns
import matplotlib.pyplot as plt
plt.figure()
sns.histplot(np.abs(corrs), kde=True)

plt.figure()
sns.histplot(np.abs(corrs_random), kde=True)


<Axes: ylabel='Count'>

# Figure 1  ISC

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5, style="whitegrid")
base_path = "D:/data/flex_ssm/"
order = ["SSM", "AP", "RAP-MUSIC"]
colors = sns.color_palette() #['#87CEEB', '#FFA07A', '#4169E1',]#  '#B22222', '#8A2BE2', '#3CB371', '#FFD700']
primers = [["(a)", "(d)"], ["(b)", "(e)"], ["(c)", "(f)"]]

filenames = [
    "results_figure-1_coarse-80_greedy.pkl",
    "results_figure-1_fine-80_greedy.pkl",
    "results_figure-1_fine-50_greedy.pkl",
    # "results_figure-21
]
titles = [
    "No Model Error",
    "Source Space Model Error",
    "Source Space +\n Conductivity Model Error",
    # "Source Space + Conductivity Error (60)",
]
subplots = [
    [2,3,1],
    [2,3,2],
    [2,3,3],
    # [2,4,4],
]
fig = plt.figure(figsize=(18, 11))

ylim = (0, 47.5)
dfs = []
for i, filename in enumerate(filenames):

    condition = "_".join(filename.split("_")[1:3])
    greedyness =filename.split("_")[3]
    fn = os.path.join(base_path, f"predictions/sim_and_preds_{condition}_{greedyness}")
    with open(fn, 'rb') as f:
        _, _, _, _, sim_info, _, _ = pkl.load(f)

    path = f"results/{filename}"
    with open(path, "rb") as f:
        results = pkl.load(f)
    df = pd.DataFrame(results)
    df["Method"] = [m  if m != "FLEX-MUSIC" else "FLEX-RAP-MUSIC" for m in df["Method"]]

    sim_info_matched = pd.concat([sim_info,]*len(set(df.Method)), axis=0)
    df["inter_source_correlations"] = sim_info_matched["inter_source_correlations"].values


    dfs.append(df)
    
    ax = plt.subplot(*subplots[i])
    sns.barplot(data=df, hue="Method", y="Mean_Localization_Error", x="inter_source_correlations", hue_order=order, palette=colors, errorbar=errorbar, estimator=estimator)
    plt.xlabel("\u03C1")
    plt.ylabel("Mean Localization Error [mm]")
    plt.ylim(*ylim)
    plt.title(f"{primers[i][0]} {titles[i]}", pad=10)
    sns.move_legend(ax, "lower right", bbox_to_anchor=(1, 1))
    ax.get_legend().remove()
    
    # EMD
    subplot_emd = subplots[i]
    subplot_emd[-1] += len(subplots)
    ax = plt.subplot(*subplot_emd)
    sns.barplot(data=df, hue="Method", y="EMD", x="inter_source_correlations", hue_order=order, palette=colors, errorbar=errorbar, estimator=estimator)
    plt.xlabel("\u03C1")
    plt.ylabel("Earth Mover's Distance [mm]")
    plt.ylim(*ylim)
    plt.title(f"{primers[i][1]} {titles[i]}", pad=10)
    sns.move_legend(ax, "lower right", bbox_to_anchor=(2, 0))
    # if i != len(filenames)-1:
    ax.get_legend().remove()

# plt.suptitle("Localization error for varying ISC")
# Add bottom bar legend
legs = fig.legend(order, loc="upper center", ncol=len(order), bbox_to_anchor=(0.5, 0.95))
for leg, col in zip(legs.legend_handles, colors):
    leg.set_color(col)
    leg.set_linewidth(5.0)

plt.tight_layout(pad=1.5)
fig.subplots_adjust(top=0.8)  # Adjust this value to suit your plot
# fig.savefig('figures/Figure_1.png', format='png', dpi=300)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/data/flex_ssm/predictions/sim_and_preds_figure-1_coarse-80_greedy.pkl'

# Figure 3: Diffusion Parameter

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5, style="whitegrid")
order = ["AP", "RAP-MUSIC", "FLEX-AP", "FLEX-RAP-MUSIC"]
colors = ['#87CEEB', '#FFA07A', '#4169E1', '#B22222', '#8A2BE2', '#3CB371', '#FFD700']
primers = [["(a)", "(d)"], ["(b)", "(e)"], ["(c)", "(f)"]]

filenames = [
    "results_figure-3_coarse-80_greedy.pkl",
    "results_figure-3_fine-80_greedy.pkl",
    "results_figure-3_fine-50_greedy.pkl",
    # "results_figure-3_fine-20.pkl",

]
titles = [
    "No Model Error",
    "Source Space Model Error",
    "Source Space Model +\n Conductivity Model Error",
    # "Source Space + Conductivity Error (60)",
]
subplots = [
    [2,3,1],
    [2,3,2],
    [2,3,3],
    # [2,4,4],
]
fig = plt.figure(figsize=(18, 11))

ylim = (0, 20)
dfs = []
for i, filename in enumerate(filenames):

    condition = "_".join(filename.split("_")[1:3])
    greedyness =filename.split("_")[3]
    fn = f"evaluation/sim_and_preds_{condition}_{greedyness}"
    with open(fn, 'rb') as f:
        _, _, _, _, sim_info, _, _ = pkl.load(f)

    path = f"results/{filename}"
    with open(path, "rb") as f:
        results = pkl.load(f)
    df = pd.DataFrame(results)
    df["Method"] = [m  if m != "FLEX-MUSIC" else "FLEX-RAP-MUSIC" for m in df["Method"]]
    sim_info_matched = pd.concat([sim_info,]*len(set(df.Method)), axis=0)
    df["diffusion_parameter"] = sim_info_matched["diffusion_parameter"].values


    dfs.append(df)
    
    ax = plt.subplot(*subplots[i])
    sns.barplot(data=df, hue="Method", y="Mean_Localization_Error", x="diffusion_parameter", hue_order=order, palette=colors, errorbar=errorbar, estimator=estimator)
    plt.xlabel("Diffusion Parameter [" + r'$ \alpha $' + "]")
    plt.ylabel("Mean Localization Error [mm]")
    plt.ylim(*ylim)
    plt.title(f"{primers[i][0]} {titles[i]}", pad=10)
    sns.move_legend(ax, "lower right", bbox_to_anchor=(1, 1))
    ax.get_legend().remove()
    
    # EMD
    subplot_emd = subplots[i]
    subplot_emd[-1] += 3
    ax = plt.subplot(*subplot_emd)
    sns.barplot(data=df, hue="Method", y="EMD", x="diffusion_parameter", hue_order=order, palette=colors, errorbar=errorbar, estimator=estimator)
    plt.xlabel("Diffusion Parameter [" + r'$ \alpha $' + "]")
    plt.ylabel("Earth Mover's Distance")
    plt.ylim(*ylim)
    plt.title(f"{primers[i][1]} {titles[i]}", pad=10)
    sns.move_legend(ax, "lower right", bbox_to_anchor=(2, 0))
    # if i != len(filenames)-1:
    ax.get_legend().remove()

# plt.suptitle("Localization error for diffusion parameters")
# Add bottom bar legend
legs = fig.legend(order, loc="upper center", ncol=len(order), bbox_to_anchor=(0.5, 0.95))
for leg, col in zip(legs.legend_handles, colors):
    leg.set_color(col)

plt.tight_layout(pad=1.5)
fig.subplots_adjust(top=0.8)  # Adjust this value to suit your plot
fig.savefig('figures/main_manuscript/Figure_3.png', format='png', dpi=300)

# Figure 3.2 Surface area prediction

In [ ]:
from funs import SurfaceCalculator


fullpath = "forward_models/128_ch_coarse_80_ratio-fwd.fif"
fwd_ico4 = mne.read_forward_solution(fullpath, verbose=0)
fwd_ico4 = mne.convert_forward_solution(fwd_ico4, force_fixed=True)


fullpath = "forward_models/128_ch_fine_80_ratio-fwd.fif"
fwd_oct6 = mne.read_forward_solution(fullpath, verbose=0)
fwd_oct6 = mne.convert_forward_solution(fwd_oct6, force_fixed=True)

get_surface_coarse = SurfaceCalculator(fwd_ico4)
get_surface_fine = SurfaceCalculator(fwd_oct6)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

sns.set(font_scale=1.1, style="whitegrid")

filenames = [
    "results_figure-3_coarse-80_greedy.pkl",
    "results_figure-3_fine-80_greedy.pkl",
    "results_figure-3_fine-50_greedy.pkl",
    # "results_figure-3_fine-20.pkl",

]
titles = [
    "No Model Error",
    "Source Space Model Error",
    "Source Space Model + Conductivity Model Error",
    # "Source Space + Conductivity Error (60)",
]
subplots = [
    [2,3,1],
    [2,3,2],
    [2,3,3],
    # [2,4,4],
]
primers = [["(a)", "(b)"], ["(c)", "(d)"], ["(e)", "(f)"]]
colors = ['#4169E1', '#B22222']
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(8, 12))
row = 0
rho = r"$\rho$"
ylim = (0, 35)
dfs = []
for i, filename in enumerate(filenames):

    condition = "_".join(filename.split("_")[1:3])
    fn = f"evaluation/sim_and_preds_{condition}_greedy.pkl"
    with open(fn, 'rb') as f:
        stc, stc_dict, _, y_test, sim_info, _, _ = pkl.load(f)
    # break
    path = f"results/{filename}"
    with open(path, "rb") as f:
        results = pkl.load(f)
    df = pd.DataFrame(results)
    df["Method"] = [m  if m != "FLEX-MUSIC" else "FLEX-RAP-MUSIC" for m in df["Method"]]
    sim_info_matched = pd.concat([sim_info,]*len(set(df.Method)), axis=0)
    df["diffusion_parameter"] = sim_info_matched["diffusion_parameter"].values

    df["index"] = np.tile(np.arange(y_test.shape[0]), 4)

    diffusion_parameters = [0.05, 0.1, 0.15]
    for ii, j in enumerate(range(0, 1500, 500)):
        true_surface_areas = np.array([get_surface_coarse(y.toarray()[0]) for y in y_test[j:j+500]])
        estimated_surface_areas = dict()
        for solver_name, stcs in stc_dict.items():
            if not "FLEX" in solver_name:
                continue
            if "_coarse" in filename:
                estimated_surface_areas[solver_name] = np.array([get_surface_coarse(abs(stc).mean(axis=-1)) for stc in stcs[j:j+500]])
            else:
                estimated_surface_areas[solver_name] = np.array([get_surface_fine(abs(stc).mean(axis=-1)) for stc in stcs[j:j+500]])


        from scipy.stats import pearsonr

        lim = (0, 15)
        

        for i_solver, (solver_name, est_surface_areas) in enumerate(estimated_surface_areas.items()):
            if not "FLEX" in solver_name:
                continue
            if solver_name == "FLEX-MUSIC":
                solver_name = "FLEX-RAP-MUSIC"
            r, p = pearsonr(true_surface_areas, est_surface_areas)
            sns.regplot(y=est_surface_areas/1000, x=true_surface_areas/1000, line_kws=dict(color="black"), scatter_kws={"color": colors[i_solver], "alpha":0.3}, ax=axes[row, i_solver])
            axes[row, i_solver].plot([lim[0], lim[1]], [lim[0], lim[1]], "--k")
            axes[row, i_solver].set_ylim(*lim)
            axes[row, i_solver].set_xlim(*lim)
            axes[row, i_solver].set_title(f"{primers[row][i_solver]} {solver_name}: {rho} = {r:.2f}")
            axes[row, i_solver].set_xlabel("True Area [cm²]")
            axes[row, i_solver].set_ylabel("Est. Area [cm²]")
            axes[row, i_solver].set_aspect('equal', 'box')
        row += 1
        # plt.suptitle(f"Diffusion Parameter = {diffusion_parameters[ii]}")
        
        
    break

plt.tight_layout()
fig.savefig('figures/main_manuscript/Figure_3_2.png', format='png', dpi=300)

# Figure 4

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5, style="whitegrid")
colors = ['#87CEEB', '#FFA07A', '#4169E1', '#B22222', '#8A2BE2', '#3CB371', '#FFD700']
primers = [["(a)", "(b)"], ["(c)", "(d)"], ["(e)", "(f)"]]

filenames = [
    "results_figure-4_coarse-80_greedy.pkl",
    "results_figure-4_fine-80_greedy.pkl",
    "results_figure-4_fine-50_greedy.pkl",
    # "results_figure-4_fine-20.pkl",

]
titles = [
    "No Model Error",
    "Source Space Model Error",
    "Source Space Model +\n Conductivity Model Error",
    # "Source Space + Conductivity Error (60)",
]
subplots = [
    [2,3,1],
    [2,3,2],
    [2,3,3],
    # [2,4,4],
]
fig = plt.figure(figsize=(18, 11))

ylim = (0, 25)
dfs = []
for i, filename in enumerate(filenames):

    condition = "_".join(filename.split("_")[1:3])
    greedyness = filename.split("_")[3]
    fn = f"evaluation/sim_and_preds_{condition}_{greedyness}"
    with open(fn, 'rb') as f:
        _, _, _, _, sim_info, _, _ = pkl.load(f)

    path = f"results/{filename}"
    with open(path, "rb") as f:
        results = pkl.load(f)
    df = pd.DataFrame(results)
    df["Method"] = [m  if m != "FLEX-MUSIC" else "FLEX-RAP-MUSIC" for m in df["Method"]]
    
    sim_info_matched = pd.concat([sim_info,]*len(set(df.Method)), axis=0)
    df["SNR"] = [int(10*np.log10(x)) for x in sim_info_matched["snr"]]


    dfs.append(df)

    ax = plt.subplot(*subplots[i])
    sns.barplot(data=df, hue="Method", y="Mean_Localization_Error", x="SNR", hue_order=order[:4], palette=colors, errorbar=errorbar, estimator=estimator)
    plt.xlabel("SNR [dB]")
    plt.ylabel("Mean Localization Error [mm]")
    plt.ylim(*ylim)
    plt.title(f"{primers[i][0]} {titles[i]}", pad=10)
    sns.move_legend(ax, "lower right", bbox_to_anchor=(1, 1))
    ax.get_legend().remove()
    
    # EMD
    subplot_emd = subplots[i]
    subplot_emd[-1] += 3
    ax = plt.subplot(*subplot_emd)
    sns.barplot(data=df, hue="Method", y="EMD", x="SNR", hue_order=order[:4], palette=colors, errorbar=errorbar, estimator=estimator)
    plt.xlabel("SNR [dB]")
    plt.ylabel("Earth Mover's Distance [mm]")
    plt.ylim(*ylim)
    plt.title(f"{primers[i][1]} {titles[i]}", pad=10)
    sns.move_legend(ax, "lower right", bbox_to_anchor=(2., 0))
    # if i != len(filenames)-1:
    ax.get_legend().remove()

# plt.suptitle("Localization error for varying SNR")
# Add bottom bar legend
legs = fig.legend(order, loc="upper center", ncol=len(order), bbox_to_anchor=(0.5, 0.95))
for leg, col in zip(legs.legend_handles, colors):
    leg.set_color(col)

plt.tight_layout(pad=1.5)
fig.subplots_adjust(top=0.8)  # Adjust this value to suit your plot
fig.savefig('figures/main_manuscript/Figure_4.png', format='png', dpi=300)

# Plot Brains Simulation

In [ ]:
import pickle as pkl
import mne
import sys; sys.path.insert(0, '../../invert')
from invert.util import pos_from_forward
from copy import deepcopy
import pandas as pd
import numpy as np
order = ["AP", "RAP-MUSIC", "FLEX-AP", "FLEX-RAP-MUSIC", "Convexity Champagne", "eLORETA", "LCMV"]

clim_greedy = dict(kind="value", pos_lims=(0.0, 0.01, 1))
clim_alt = dict(kind="value", pos_lims=(0.5, 0.75, 1))
pp = dict(
    surface='inflated', 
    hemi='both', 
    background="white", 
    colorbar=False, 
    time_viewer=False,
    cortex="low_contrast",
    verbose=0)


fn = f"evaluation/sim_and_preds_figure-1_coarse-80_non-greedy.pkl"
with open(fn, 'rb') as f:
    stc, stc_dict, x_test, y_test, _, _, _ = pkl.load(f)

fn = f"evaluation/sim_and_preds_figure-1_coarse-80_greedy.pkl"
with open(fn, 'rb') as f:
    _, stc_dict2, _, _, _, _, _ = pkl.load(f)
    
stc_dict.update(stc_dict2)
stc_dict["FLEX-RAP-MUSIC"] = stc_dict["FLEX-MUSIC"]
del stc_dict["FLEX-MUSIC"]
stc_dict = {key: stc_dict[key] for key in order}

fwd = mne.read_forward_solution("forward_models/128_ch_coarse_80_ratio-fwd.fif", verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
pos = pos_from_forward(fwd)
source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]

fullpath = "forward_models/128_ch_info.pkl"
with open(fullpath, "rb") as f:
    info = pkl.load(f)

samples = [6, 1006, 237, 737]
hemis = ["rh", "rh", "lh", "lh"]
views = ["lateral", "lateral","lateral", "lateral"]

imgs = []
colorbars = []
names = []
solver = "??"
for i_sample, sample in enumerate(samples):
    
    tmin = 0
    tstep = 1/1000
    subject = "fsaverage"

    evoked = mne.EvokedArray(x_test[sample].T, info, tmin=0)
    # evoked.plot_joint()
    stc = mne.SourceEstimate(y_test[sample].T, vertices, tmin=tmin, tstep=tstep, 
                            subject=subject, verbose=0)
    first_sample = stc.data[:, 0].toarray()
    first_sample /= np.max(abs(first_sample))
    stc.data = abs(np.tile(first_sample, (1,20)))
    pp["views"] = views[i_sample]
    pp["hemi"] = hemis[i_sample]
    brain = stc.plot(**pp, brain_kwargs=dict(title="True"), clim=clim_greedy)
    pp["colorbar"] = True
    brain_cb = stc.plot(**pp, brain_kwargs=dict(title=solver), clim=clim_greedy)
    pp["colorbar"] = False

    img = brain.screenshot()
    colorbar = brain_cb.screenshot()
    brain.close()
    brain_cb.close()
    imgs.append( img )
    colorbars.append( colorbar )
    names.append("Ground Truth")
    
    for solver, stc_list in stc_dict.items():
        first_sample = stc_list[sample][:, 0].toarray()
        first_sample /= np.max(abs(first_sample))
        stc.data = np.tile(first_sample, (1,20))
        if solver in ['eLORETA', 'LCMV'] :
            clim = clim_alt
        else:
            clim = clim_greedy

        pp["views"] = views[i_sample]
        pp["hemi"] = hemis[i_sample]
        brain = abs(stc).plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
        pp["colorbar"] = True
        brain_cb = abs(stc).plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
        pp["colorbar"] = False

        img = brain.screenshot()
        colorbar = brain_cb.screenshot()
        
        brain.close()
        brain_cb.close()

        imgs.append( img )
        colorbars.append(colorbar)
        names.append(solver)

In [ ]:
n_solvers = len(stc_dict)+1
n_imgs = len(imgs)
n_cols = int(n_imgs / n_solvers)
fig, axes = plt.subplots(nrows=n_solvers, ncols=n_cols, figsize=(8, 12))

# sample_names = ["(a) Example 1\nDiscrete", "(b) Example 1\nExtended", "(c) Example 2\nDiscrete", "(d) Example 2\nExtended"]
sample_names = ["(a) Discrete", "(b) Extended", "(c) Discrete", "(d) Extended"]
solver_names = names[:n_solvers]
for i_col in range(n_cols):
    for i_row in range(n_solvers):
        idx = i_col * n_solvers + i_row
        ax = axes[i_row, i_col]

        img = imgs[idx]
        left = np.where((img-255).sum(axis=(0,-1))!=0)[0][0]
        right = np.where((img-255).sum(axis=(0,-1))!=0)[0][-1]

        upper = np.where((img-255).sum(axis=(1,-1))!=0)[0][0]
        lower = np.where((img-255).sum(axis=(1,-1))!=0)[0][-1]
        img = img[upper:lower, left:right]
        
        ax.imshow(img)
        ax.axis('off')

        if i_row == 0:
            ax.set_title(f"{sample_names[i_col]}")

plt.tight_layout(pad=1)
plt.subplots_adjust(wspace=0.1, hspace=0, left=0.2, right=1, bottom=0, top=0.85)  # Adjust subplot parameters to make it super tight




# Row Titles
row_titles = solver_names
for ax, row_title in zip(axes[:,0], row_titles):
    if row_title == "FLEX-RAP-MUSIC":
        row_title = "FLEX-\nRAP-MUSIC"
    if " " in row_title:
        row_title = row_title.replace(" ", "\n")
        
    
    # Compute the vertical center of the ax object
    bbox = ax.get_position()
    y_center = (bbox.y0 + bbox.y1) / 2
    fig.text(0.02, y_center, row_title, va='center', ha='left', fontsize=16)

# Column Titles
# Compute the horizontal center between column 0 and 1
bbox0 = axes[0, 0].get_position()
bbox1 = axes[0, 1].get_position()
x_center_01 = (bbox0.x1 + bbox1.x0) / 2
fig.text(x_center_01, 0.9, 'Example 1', va='bottom', ha='center', fontsize=16, transform=fig.transFigure)

# Compute the horizontal center between column 2 and 3 (note: we don't have a column 3, so it's between 1 and 2 in this example)
bbox2 = axes[0, 2].get_position()
bbox3 = axes[0, 3].get_position()
x_center_23 = (bbox2.x1 + bbox3.x0) / 2
fig.text(x_center_23, 0.9, 'Example 2', va='bottom', ha='center', fontsize=16, transform=fig.transFigure)


fig.savefig(f'figures/main_manuscript/Brain_Plots_Simulation.png', format='png', dpi=600)

In [ ]:
import matplotlib.pyplot as plt

n_samples = len(samples)  
n_views = len(views)
n_solvers = len(imgs) // n_views // n_samples
solver_names = ["Ground Truth",] + list(stc_dict.keys())
for i_samp in range(n_samples):
    fig = plt.figure(figsize=(8, 13))
    for i in range(n_solvers):
        for j in range(n_views):
            print(i*n_views+j+1)
            plt.subplot(n_solvers, n_views, i*n_views+j+1)
            plt.imshow(imgs[i_samp*n_views*n_solvers + i*n_views+j])
            plt.axis('off')  # Turn off the axes, tick labels, and ticks
            if j == 0:
                if solver_names[i] == "Convexity Champagne":
                    plt.title("Champagne")
                else:
                    plt.title(solver_names[i])

            

    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)  # Adjust subplot parameters to make it super tight
    plt.tight_layout(pad=0.1)
    # fig.savefig(f'figures/main_manuscript/Brain_Plots_{i_samp}.png', format='png', dpi=600)
    


In [ ]:
for i_samp, sample in enumerate(samples):
    evoked = mne.EvokedArray(x_test[sample].T, info)
    fig = evoked.plot_joint(times=[0.01, 0.02, 0.03, 0.04])
    fig.savefig(f'figures/main_manuscript/Evoked_Plots_{i_samp}.png', format='png', dpi=600)


In [ ]:
fn = r"results\results_figure-1_coarse-80_greedy.pkl"
with open(fn, "rb") as f:
    results = pkl.load(f)
df = pd.DataFrame(results)
emd_diff = df[df.Method=="FLEX-AP"].EMD.values - df[df.Method=="FLEX-MUSIC"].EMD.values

idc = np.argsort(emd_diff)
idc = idc[ (idc>499) & (idc < 1000)  ]
print(idc)

pptemp = dict(surface="inflated", hemi="both", verbose=0)
extent_jump = 500

# i = idc[0] - extent_jump
i = 737  # 718

stc.data = y_test[i].toarray().T
abs(stc).plot(brain_kwargs=dict(title=f"GT {i}"), **pptemp)

stc.data = stc_dict["AP"][i].toarray()
abs(stc).plot(brain_kwargs=dict(title=f"AP {i}"), **pptemp)

stc.data = stc_dict["FLEX-AP"][i].toarray()
abs(stc).plot(brain_kwargs=dict(title=f"FLAP {i}"), **pptemp)

stc.data = stc_dict["RAP-MUSIC"][i].toarray()
abs(stc).plot(brain_kwargs=dict(title=f"RAP-MUSIC {i}"), **pptemp)

stc.data = stc_dict["FLEX-RAP-MUSIC"][i].toarray()
abs(stc).plot(brain_kwargs=dict(title=f"FLEX-RAP-MUSIC {i}"), **pptemp)
# for i in idc[10:20]:
#     stc.data = y_test[i].toarray().T
#     abs(stc).plot(brain_kwargs=dict(title=f"GT {i}"), **pptemp)

#     # stc.data = stc_dict["AP"][i].toarray()
#     # abs(stc).plot(brain_kwargs=dict(title=f"AP {i}"), **pptemp)

#     # stc.data = stc_dict["FLEX-AP"][i].toarray()
#     # abs(stc).plot(brain_kwargs=dict(title=f"FLAP {i}"), **pptemp)

#     # stc.data = stc_dict["RAP-MUSIC"][i].toarray()
#     # abs(stc).plot(brain_kwargs=dict(title=f"RAP-MUSIC {i}"), **pptemp)

#     # stc.data = stc_dict["FLEX-RAP-MUSIC"][i].toarray()
#     # abs(stc).plot(brain_kwargs=dict(title=f"FLEX-RAP-MUSIC {i}"), **pptemp)
    




# Case study: FLEX for 0 extent has lower MLE

In [ ]:
import pickle as pkl
import mne
import sys; sys.path.insert(0, '../../invert')
from invert.util import pos_from_forward
from copy import deepcopy
import pandas as pd
import numpy as np

pp = dict(
    surface='inflated', 
    hemi='both', 
    background="white", 
    colorbar=False, 
    time_viewer=False, 
    smoothing_steps=5, 
    colormap="Reds",
    cortex="low_contrast",
    verbose=0)
hemis = ["lh", "lh"]
views = ["lateral", "medial"]

fn = f"evaluation/sim_and_preds_figure-1_fine-80_greedy.pkl"
with open(fn, 'rb') as f:
    stc, stc_dict, x_test, y_test, _, _, _ = pkl.load(f)

fn = r"results\results_figure-1_fine-80_greedy.pkl"
with open(fn, 'rb') as f:
    results = pkl.load(f)
    
fwd_fine = mne.read_forward_solution("forward_models/128_ch_fine_80_ratio-fwd.fif", verbose=0)
fwd_fine = mne.convert_forward_solution(fwd_fine, force_fixed=True)
pos_fine = pos_from_forward(fwd_fine)

fwd_coarse = mne.read_forward_solution("forward_models/128_ch_coarse_80_ratio-fwd.fif", verbose=0)
fwd_coarse = mne.convert_forward_solution(fwd_coarse, force_fixed=True)
pos_coarse = pos_from_forward(fwd_coarse)


fullpath = "forward_models/128_ch_info.pkl"
with open(fullpath, "rb") as f:
    info = pkl.load(f)


# Get the idc where MLE is most different
df = pd.DataFrame(results)
df["index"] = np.tile(np.arange(1500), 8)
# Select only zero-extent ones
df = df[df["Active_True"]==2] 
df_flex = df[df["Method"]=="FLEX-AP"]
mle_flex = df_flex["Mean_Localization_Error"].values 

df_ap = df[df["Method"]=="AP"]
mle_ap = df_ap["Mean_Localization_Error"].values 

diff = mle_flex - mle_ap
# idc = np.argsort(diff)
diff_range = (-99, -0.5)
n_verts = pos_fine.shape[0]
imgs = []
n_imgs = 0
idc = [154, 401]  # , 324
for idx in idc:
    # if diff[idx] < diff_range[0] or diff[idx] > diff_range[1] or mle_flex[idx] > 6 or idx in [81, 0, 442, 70, 411, 168, 191, 290, 3]:  # 154, 401, 324
    #     continue
    # else:
    #     print(f"idx {idx}: diff={diff[idx]:.1f}, mle_flex={mle_flex[idx]}")
    
    maxima_idc = np.where(y_test[idx].toarray()[0, :]!=0)[0]
    positions = pos_coarse[maxima_idc, :]

    # true_positions = []
    true_verts = []
    for p in positions:
        vert = np.argmin(np.sum((pos_fine-p)**2, axis=1))
        true_verts.append(vert)
        # true_positions.append(pos_fine[vert, :])
    
    imgs_AP = []
    imgs_FLEX = []

    for hemi, view in zip(hemis, views):
        pp["views"] = view
        pp["hemi"] = hemi
        # AP
        stc.data = abs(stc_dict["AP"][idx].toarray())
        clim = dict(kind="value", lims=(0, 0, stc.data[:, 0].max()/10))
        brain = stc.plot(clim=clim, **pp)
        
        for i, pos in enumerate(positions):#
            if pos[0] < 0:  # if ground truth is left
                vert = stc.lh_vertno[true_verts[i]]
                if hemi == "both":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=0.75, hemi="lh")
                elif hemi == "lh":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=0.75)
            else: # if ground truth is right
                vert = stc.rh_vertno[true_verts[i]-4098]
                if hemi == "both":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=0.75, hemi="rh")
                elif hemi == "rh":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=0.75)
        # if view == "lateral":
        #     brain.show_view(view="lateral", azimuth=215)

        img_AP = brain.screenshot()
        brain.close()

        # remove white borders from img
        left = np.where((img_AP-255).sum(axis=(0,-1))!=0)[0][0]
        right = np.where((img_AP-255).sum(axis=(0,-1))!=0)[0][-1]
        upper = np.where((img_AP-255).sum(axis=(1,-1))!=0)[0][0]
        lower = np.where((img_AP-255).sum(axis=(1,-1))!=0)[0][-1]
        img_AP = img_AP[upper:lower, left:right]

        imgs_AP.append(img_AP)
        

        # FLEX
        stc.data = abs(stc_dict["FLEX-AP"][idx].toarray())
        clim = dict(kind="value", lims=(0, 0, stc.data[:, 0].max()/10))
        brain = stc.plot(clim=clim, **pp)
        
        for i, pos in enumerate(positions):#
            if pos[0] < 0:  # if ground truth is left
                vert = stc.lh_vertno[true_verts[i]]
                if hemi == "both":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=1, hemi="lh")
                elif hemi == "lh":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=1)
            else: # if ground truth is right
                vert = stc.rh_vertno[true_verts[i]-4098]
                if hemi == "both":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=1, hemi="rh")
                elif hemi == "rh":
                    brain.add_foci(vert, coords_as_verts=True, color="green", alpha=1)
        
        # if view == "lateral":
        #     brain.show_view(view="lateral", azimuth=215)
        img_FLEX = brain.screenshot()
        brain.close()
        
        # remove white borders from img
        left = np.where((img_FLEX-255).sum(axis=(0,-1))!=0)[0][0]
        right = np.where((img_FLEX-255).sum(axis=(0,-1))!=0)[0][-1]
        upper = np.where((img_FLEX-255).sum(axis=(1,-1))!=0)[0][0]
        lower = np.where((img_FLEX-255).sum(axis=(1,-1))!=0)[0][-1]
        img_FLEX = img_FLEX[upper:lower, left:right]

        imgs_FLEX.append(img_FLEX)
    imgs.append( (imgs_AP, imgs_FLEX) )
    
    n_imgs += 1
    if n_imgs == 3:
        break



In [ ]:
n_samples = len(imgs)
n_solvers = 2
n_views = len(views)

fig, axes = plt.subplots(nrows=int(n_samples*n_solvers), ncols=n_views, figsize=(10, 12))

for i_sample,  (imgs_AP, imgs_FLEX) in enumerate(imgs):
    i_sample *= 2
    for i_view in range(n_views):
        axes[i_sample, i_view].imshow(imgs_AP[i_view])
        axes[i_sample, i_view].set_axis_off()
        # if i_view==0:
        #     axes[i_sample, i_view].set_title("AP")
        if i_view == 0:
            axes[i_sample, i_view].text(-0.21, 0.6, "AP", ha='center', va='center', transform=axes[i_sample, i_view].transAxes, fontsize=24)


        axes[i_sample+1, i_view].imshow(imgs_FLEX[i_view])
        axes[i_sample+1, i_view].set_axis_off()
        if i_view == 0:
            axes[i_sample+1, i_view].text(-0.21, 0.6, "FLEX-AP", ha='center', va='center', transform=axes[i_sample+1, i_view].transAxes, fontsize=24)



plt.subplots_adjust(wspace=0.3, hspace=0, left=0, right=1, bottom=0, top=1)  # Adjust subplot parameters to make it super tight
plt.tight_layout(pad=1)
fig.savefig(f'figures/main_manuscript/Brain_Plots_source_model_error.png', format='png', dpi=600)

In [ ]:
# for idx in idc:
#     if diff[idx] < diff_range[0] or diff[idx] > diff_range[1] or mle_flex[idx] > 6 or idx in [81, 0, 442, 70, 411, 168, 191, 290, 3]:  # 154, 401, 324
#         continue

#     print(idx)

stc.data = abs(stc_dict["AP"][324].toarray())
clim = dict(kind="value", lims=(0, 0, stc.data[:, 0].max()/2))
brain = stc.plot(surface="inflated", hemi="lh", views="medial", verbose=0, cortex="low_contrast", clim=clim)
brain.show_view(view="lateral", azimuth=250)
# break